In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
from io import StringIO
import os

In [40]:
# setting the years needed and url to look at player
years = list(range(2017, 2026))
url_start = 'https://www.basketball-reference.com/players/j/jokicni01/gamelog/{}'

In [132]:
# looks up the corresponding year and creates/overwrites the html file in my JokicGameLogs folder
for year in years:   
    url = url_start.format(year)
    response = requests.get(url)
    # to write a file for each 'x'
    with open("JokicGameLogs/{}.html".format(year), "w+", encoding="utf-8") as f:
        f.write(response.text)

In [142]:
dfs = {}
for year in years:
    with open(f"JokicGameLogs/{year}.html", encoding="utf-8") as f:
        page = f.read()
        soup = BeautifulSoup(page, "html.parser")
        stats_table = soup.find(id="pgl_basic")
        stats_df = pd.read_html(StringIO(str(stats_table)))[0]
        dfs[f'{year}'] = stats_df

In [143]:
# Drop rows where the index + 1 is divisible by 20 (21, 41, 61, ...)
for year, df in dfs.items():
    if 20 in df.index:
        df = df.drop(20)
    if 41 in df.index:
        df = df.drop(41)
    if 62 in df.index:
        df = df.drop(62)
    if 83 in df.index:
        df = df.drop(83)
    dfs[year] = df

In [145]:
for year, df in dfs.items():
    # drop the 'Rk' column so we can use our index col instead
    if 'Rk' in df.columns:
        df = df.drop(columns=['Rk'])
    # drop null rows
    df = df.dropna(subset=['G'])
    dfs[year] = df

In [146]:
dfs['2025']

,G,Date,Age,Tm,Unnamed: 5,Opp,Unnamed: 7,GS,MP,FG,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,1,2024-10-24,29-248,DEN,NaN,OKC,L (-15),1,35:12,6,...,8,12,13,2,1,3,3,16,21.7,-9
1,2,2024-10-26,29-250,DEN,NaN,LAC,L (-5),1,36:46,14,...,6,9,4,2,1,5,2,41,31.2,+7
2,3,2024-10-28,29-252,DEN,@,TOR,W (+2),1,43:41,18,...,7,10,4,1,2,3,2,40,33.5,+9
3,4,2024-10-29,29-253,DEN,@,BRK,W (+5),1,40:31,9,...,12,18,16,0,1,1,3,29,38.1,+8
4,5,2024-11-01,29-256,DEN,@,MIN,L (-3),1,39:42,8,...,7,9,13,3,1,3,1,26,30.1,+6
5,6,2024-11-02,29-257,DEN,NaN,UTA,W (+26),1,29:42,10,...,11,16,9,1,0,5,1,27,27.1,+31
6,7,2024-11-04,29-259,DEN,NaN,TOR,W (+2),1,38:12,10,...,9,14,13,1,2,7,2,28,26.1,-1
7,8,2024-11-06,29-261,DEN,NaN,OKC,W (+2),1,39:29,9,...,13,20,16,2,2,5,2,23,29.4,+8
8,9,2024-11-08,29-263,DEN,NaN,MIA,W (+13),1,40:00,11,...,9,11,14,2,0,5,2,30,35.4,+26
9,10,2024-11-10,29-265,DEN,NaN,DAL,W (+2),1,38:01,13,...,10,18,15,3,0,4,2,37,44.8,+13
